# Ocean Heat Content

$OHC = \Delta T \times c_p \times \rho \times V$

unit conversion to SI
- $c_p$: erg/g/K = 1e-7J / 1e-3kg / K = 1e-4 J/kg/K $\rightarrow$ 3996 J/kg/K
- $\rho$: g/cm^3 = 1e3 kg/m^3

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import OHC_integrals, trend_global_levels, replace_OHC_year
from maps import map_robinson, map_eq_earth
from grid import create_dz_mean, create_tdepth
from paths import CESM_filename, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rcp, path_results
from regions import boolean_mask, regions_dict
from plotting import discrete_cmap, shifted_color_map
from constants import cp_sw, spy, km, spd, Jpy_to_Wpsm
from OHC_plots import plot_global_integrals, plot_global_integrals_detr, plot_global_integrals_diff, plot_levels_trend
from OHC_plots import Hovmoeller_global_depth, Hovmoeller_basins_depth
from timeseries import IterateOutputCESM, ncfile_list, lowpass, chebychev
from xr_integrate import  xr_vol_int
from xr_DataArrays import create_xr_DataArray, xr_DZ, xr_AREA, xr_HTN, xr_LATS
from xr_regression import xr_linear_trend, xr_linear_trends_2D, zonal_trend, zonal_levels_trend,\
                          ocn_field_regression, xr_lintrend, xr_quadtrend

# CESM
OHC integrals are calculated via the following function in approx. 10 min per year `OHC_integrals(domain='ocn', run='ctrl')`

In [ ]:
dz_mean = create_dz_mean(domain='ocn')
tdepth = create_tdepth('ocn')

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK
MASK.plot(vmin=0);

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_ctrl.nc'  , decode_times=False)
ctrl_A = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_ctrl.nc', decode_times=False)
ctrl_P = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_ctrl.nc' , decode_times=False)
ctrl_I = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_ctrl.nc'  , decode_times=False)
ctrl_M = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_ctrl.nc' , decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_ctrl.nc', decode_times=False)
ctrl_N = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_ctrl.nc'  , decode_times=False)

# something is still wrong in CTRL year 205
ctrl   = replace_OHC_year(ctrl  , 205)
ctrl_A = replace_OHC_year(ctrl_A, 205)
ctrl_P = replace_OHC_year(ctrl_P, 205)
ctrl_I = replace_OHC_year(ctrl_I, 205)
ctrl_M = replace_OHC_year(ctrl_M, 205)
ctrl_S = replace_OHC_year(ctrl_S, 205)
ctrl_N = replace_OHC_year(ctrl_N, 205)

rcp    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_rcp.nc'   , decode_times=False)
rcp_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_rcp.nc' , decode_times=False)
rcp_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_rcp.nc'  , decode_times=False)
rcp_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_rcp.nc'   , decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_rcp.nc'  , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_rcp.nc' , decode_times=False)
rcp_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_rcp.nc'   , decode_times=False)

lpd    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_lpd.nc'   , decode_times=False)
lpd_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_lpd.nc' , decode_times=False)
lpd_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_lpd.nc'  , decode_times=False)
lpd_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_lpd.nc'   , decode_times=False)
lpd_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_lpd.nc'  , decode_times=False)
lpd_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_lpd.nc' , decode_times=False)
lpd_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_lpd.nc'   , decode_times=False)

lpi    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_lpi.nc'   , decode_times=False)
lpi_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_lpi.nc' , decode_times=False)
lpi_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_lpi.nc'  , decode_times=False)
lpi_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_lpi.nc'   , decode_times=False)
lpi_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_lpi.nc'  , decode_times=False)
lpi_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_lpi.nc' , decode_times=False)
lpi_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_lpi.nc'   , decode_times=False)

In [ ]:
print(len(ctrl.time), len(rcp.time))

## global integral

In [ ]:
OHC_global_init = ctrl.OHC_global[100].item()

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(12,10), sharex=True)
for i in range(3):
    ax[i].axhline(0, c='k', lw=.5)
    ax[i].tick_params(labelsize=14)
    
ax[0].set_ylabel('adjusting OHC [J]', fontsize=14)

ax[0].plot(ctrl.time/365 + 1850, ctrl.OHC_global - ctrl.OHC_global[-200])
ax[0].plot(rcp .time/365 +  200, rcp .OHC_global - rcp .OHC_global[0]   )
ax[0].plot(lpd .time/365 + 1350, lpd .OHC_global - lpd .OHC_global[-200])
ax[0].plot(lpi .time/365 - 1600, lpi .OHC_global - lpi .OHC_global[-200])

ax[1].set_ylabel('quad. detrended OHC [ZJ]', fontsize=14)
ax[1].plot(ctrl.time/365 + 1850, (ctrl.OHC_global - xr_quadtrend(ctrl.OHC_global))/1e21, lw=.5)
ax[1].plot(rcp .time/365 +  200, (rcp .OHC_global - xr_quadtrend(rcp .OHC_global))/1e21, lw=.5)
ax[1].plot(lpd .time/365 + 1350, (lpd .OHC_global - xr_quadtrend(lpd .OHC_global))/1e21, lw=.5)
ax[1].plot(lpi .time/365 - 1600, (lpi .OHC_global - xr_quadtrend(lpi .OHC_global))/1e21, lw=.5)

ax[1].plot(ctrl.time/365 + 1850, lowpass(ctrl.OHC_global - xr_quadtrend(ctrl.OHC_global), 13)/1e21, c='C0')
ax[1].plot(rcp .time/365 +  200, lowpass(rcp .OHC_global - xr_quadtrend(rcp .OHC_global), 13)/1e21, c='C1')
ax[1].plot(lpd .time/365 + 1350, lowpass(lpd .OHC_global - xr_quadtrend(lpd .OHC_global), 13)/1e21, c='C2')
ax[1].plot(lpi .time/365 - 1600, lowpass(lpi .OHC_global - xr_quadtrend(lpi .OHC_global), 13)/1e21, c='C3')
ax[1].text(1800, -50, '13 year lowpass filter', fontsize=14)

ax[2].set_ylabel(r'$\Delta$OHC [W m$^{-2}$]', fontsize=14)
ax[2].plot(ctrl.time/365 + 1850, (ctrl.OHC_global - ctrl.OHC_global.shift(time=1))*Jpy_to_Wpsm, lw=.5, alpha=.5)
ax[2].plot(rcp .time/365 +  200, (rcp .OHC_global - rcp .OHC_global.shift(time=1))*Jpy_to_Wpsm-1, lw=.5, alpha=.5)
ax[2].plot(lpd .time/365 + 1350, (lpd .OHC_global - lpd .OHC_global.shift(time=1))*Jpy_to_Wpsm, lw=.5, alpha=.5)
ax[2].plot(lpi .time/365 - 1600, (lpi .OHC_global - lpi .OHC_global.shift(time=1))*Jpy_to_Wpsm, lw=.5, alpha=.5)

ax[2].plot(ctrl.time[:-1]/365 + 1850, lowpass((ctrl.OHC_global - ctrl.OHC_global.shift(time=1)).dropna(dim='time'), 13)*Jpy_to_Wpsm, c='C0')
ax[2].plot(rcp .time[:-1]/365 +  200, lowpass((rcp .OHC_global - rcp .OHC_global.shift(time=1)).dropna(dim='time'), 13)*Jpy_to_Wpsm-1, c='C1')
ax[2].plot(lpd .time[:-1]/365 + 1350, lowpass((lpd .OHC_global - lpd .OHC_global.shift(time=1)).dropna(dim='time'), 13)*Jpy_to_Wpsm, c='C2')
ax[2].plot(lpi .time[:-1]/365 - 1600, lowpass((lpi .OHC_global - lpi .OHC_global.shift(time=1)).dropna(dim='time'), 13)*Jpy_to_Wpsm, c='C3')
ax[2].text(2180, 1, 'offset\nby -1', fontsize=14)

ax[2].set_xticks(np.arange(0,2400,200))
ax[2].set_xlim((-50, 2350))
ax[2].set_xlabel('time [years]', fontsize=14)
f.align_ylabels()
f.tight_layout()
f.subplots_adjust(left=0.08)
plt.savefig(f'{path_results}/OHC/OHC_global_all_runs')

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl.time/365     ,(ctrl.OHC_global-xr_quadtrend(ctrl.OHC_global))/1e21, lw=2, label='CTRL quad. detrended')
plt.plot(rcp .time/365-1800,(rcp .OHC_global-xr_quadtrend(rcp .OHC_global))/1e21, lw=2, label='RCP quad. detrended')
plt.plot(ctrl.time/365     ,(ctrl.OHC_global-ctrl.OHC_global.shift(time=1))/1e21, ls=':', lw=2, c=f'C{0}', label='$\Delta$OHC CTRL')
plt.plot(rcp .time/365-1800,(rcp .OHC_global-rcp .OHC_global.shift(time=1))/1e21, ls=':', lw=2, c=f'C{1}', label='$\Delta$OHC RCP')
# plt.ylim((-25,25))
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('detrended OHC, OHC change [ZJ]', fontsize=16)
plt.legend(loc=4, fontsize=16, frameon=False, ncol=2)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_global_timeseries_detrended')

In [ ]:
# detrended rcp timeseries to find location of hiatus
dss = [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S]
plot_global_integrals(     dss, 'ctrl')
plot_global_integrals_diff(dss, 'ctrl')
plot_global_integrals_detr(dss, 'ctrl')

dss = [rcp, rcp_A, rcp_P, rcp_I, rcp_S]
plot_global_integrals(     dss, 'rcp')
plot_global_integrals_diff(dss, 'rcp')
plot_global_integrals_detr(dss, 'rcp')

dss = [lpd, lpd_A, lpd_P, lpd_I, lpd_S]
plot_global_integrals(     dss, 'lpd')
plot_global_integrals_diff(dss, 'lpd')
plot_global_integrals_detr(dss, 'lpd')

dss = [lpi, lpi_A, lpi_P, lpi_I, lpi_S]
plot_global_integrals(     dss, 'lpi')
plot_global_integrals_diff(dss, 'lpi')
plot_global_integrals_detr(dss, 'lpi')

In [ ]:
from synthesis import TimeSeriesSynthesis

In [ ]:
TSS = TimeSeriesSynthesis()
dss = [(ctrl.OHC_global - xr_quadtrend(ctrl.OHC_global))/1e21, 
       (rcp .OHC_global - xr_quadtrend(rcp .OHC_global))/1e21, 
       (lpd .OHC_global - xr_quadtrend(lpd .OHC_global))/1e21, 
       (lpi .OHC_global - xr_quadtrend(lpi .OHC_global))/1e21]

dss2 = [ctrl.OHC_global/1e21, 
        rcp .OHC_global/1e21, 
        lpd .OHC_global/1e21, 
        lpi .OHC_global/1e21]

TSS.plot_all_spectra(dss)

In [ ]:
TSS.print_all_autocorrelations(timeseries=dss)

In [ ]:
TSS.print_pairwise_homoscedasticity(fields=tuple(dss))

## Levels

In [ ]:
ctrl_levels_trend_G = trend_global_levels(ctrl)
ctrl_levels_trend_A = trend_global_levels(ctrl_A)
ctrl_levels_trend_P = trend_global_levels(ctrl_P)
ctrl_levels_trend_I = trend_global_levels(ctrl_I)
ctrl_levels_trend_S = trend_global_levels(ctrl_S)
ctrl_levels_trend_M = trend_global_levels(ctrl_M)

rcp_levels_trend_G  = trend_global_levels(rcp)
rcp_levels_trend_A  = trend_global_levels(rcp_A)
rcp_levels_trend_P  = trend_global_levels(rcp_P)
rcp_levels_trend_I  = trend_global_levels(rcp_I)
rcp_levels_trend_S  = trend_global_levels(rcp_S)
rcp_levels_trend_M  = trend_global_levels(rcp_M)

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.10,0.13,.88,.85])
plt.tick_params(labelsize=14)
plt.axvline(0, c='k', lw=.5)
plt.plot(ctrl_levels_trend_G, -tdepth/1e3, lw=3, label='CTRL')
plt.plot(rcp_levels_trend_G , -tdepth/1e3, lw=3, label='RCP')
plt.xlabel('OHC trend [J/m/yr]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_trend_depth')

In [ ]:
das = [ctrl_levels_trend_G,
       ctrl_levels_trend_A,
       ctrl_levels_trend_P,
       ctrl_levels_trend_I,
       ctrl_levels_trend_S,
       ctrl_levels_trend_M,
      ]
plot_levels_trend(das=das, run='ctrl')

das = [rcp_levels_trend_G,
       rcp_levels_trend_A,
       rcp_levels_trend_P,
       rcp_levels_trend_I,
       rcp_levels_trend_S,
       rcp_levels_trend_M,
      ]
plot_levels_trend(das=das, run='rcp')

das = [rcp_levels_trend_G-ctrl_levels_trend_G,
       rcp_levels_trend_A-ctrl_levels_trend_A,
       rcp_levels_trend_P-ctrl_levels_trend_P,
       rcp_levels_trend_I-ctrl_levels_trend_I,
       rcp_levels_trend_S-ctrl_levels_trend_S,
       rcp_levels_trend_M-ctrl_levels_trend_M,
      ]
plot_levels_trend(das=das, run='rcp-ctrl')

In [ ]:
Hovmoeller_global_depth(ctrl, detrend='lin' , fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_global_ctrl')
Hovmoeller_global_depth(rcp , detrend='lin' , fn=None)
Hovmoeller_global_depth(rcp , detrend='quad', fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_global_rcp')

In [ ]:
dss_ctrl = [ctrl_S, ctrl_A, ctrl_P, ctrl_I]
dss_rcp  = [rcp_S, rcp_A, rcp_P, rcp_I]
Hovmoeller_basins_depth(dss_ctrl, detrend='lin' , fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_basins_ctrl')
Hovmoeller_basins_depth(dss_rcp , detrend='lin' , fn=None) 
Hovmoeller_basins_depth(dss_rcp , detrend='quad', fn=f'{path_results}/OHC/OHC_Hovmoeller_depth_basins_rcp' )

## spatial trends of vertically integrated OHC

In [ ]:
%%time
# 7 min for all 9, ignoring intercept values
OHC_vert_trend_c1 , interc = ocn_field_regression(xa=ctrl.OHC_vertical[:101,:,:], run='ctrl') 
OHC_vert_trend_c2 , interc = ocn_field_regression(xa=ctrl.OHC_vertical[101:,:,:], run='ctrl') 
OHC_vert_trend_rcp, interc = ocn_field_regression(xa=rcp .OHC_vertical[:,:,:]   , run='rcp' )

(OHC_vert_trend_c1 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_c1.nc' )
(OHC_vert_trend_c2 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_c2.nc' )
(OHC_vert_trend_rcp/spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_rcp.nc')

OHC_vert_trend_c1 , interc = ocn_field_regression(xa=ctrl.OHC_vertical_above_100m[:101,:,:], run='ctrl') 
OHC_vert_trend_c2 , interc = ocn_field_regression(xa=ctrl.OHC_vertical_above_100m[101:,:,:], run='ctrl') 
OHC_vert_trend_rcp, interc = ocn_field_regression(xa=rcp .OHC_vertical_above_100m[:,:,:]   , run='rcp' )

(OHC_vert_trend_c1 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c1.nc' )
(OHC_vert_trend_c2 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c2.nc' )
(OHC_vert_trend_rcp/spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_a_rcp.nc')

OHC_vert_trend_c1 , interc = ocn_field_regression(xa=ctrl.OHC_vertical_below_100m[:101,:,:], run='ctrl') 
OHC_vert_trend_c2 , interc = ocn_field_regression(xa=ctrl.OHC_vertical_below_100m[101:,:,:], run='ctrl') 
OHC_vert_trend_rcp, interc = ocn_field_regression(xa=rcp .OHC_vertical_below_100m[:,:,:]   , run='rcp' )

(OHC_vert_trend_c1 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c1.nc' )
(OHC_vert_trend_c2 /spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c2.nc' )
(OHC_vert_trend_rcp/spd).to_netcdf(f'{path_samoc}/OHC/OHC_trend_vert_int_b_rcp.nc')

In [ ]:
OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_rcp.nc', decode_times=False)

OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_rcp.nc', decode_times=False)

OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_rcp.nc', decode_times=False)

In [ ]:
label  = r'OHC trend [W/m$^2$]'
domain = 'ocn_T'
minv   = -2
maxv   = 6
cmap   = discrete_cmap(16, shifted_color_map(mpl.cm.RdBu_r, start=.33, midpoint=0.5, stop=1., name='shrunk'))

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_c1'
text1 = 'C1'
xa = OHC_vert_trend_c1
f, ax = map_eq_earth(xa=xa, domain=domain, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=fn, text1=text1)

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_c2'
text1 = 'C2'
xa = OHC_vert_trend_c2
f, ax = map_eq_earth(xa=xa, domain=domain, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=fn, text1=text1)

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_rcp'
text1 = 'RCP'
xa = OHC_vert_trend_rcp
f, ax = map_eq_earth(xa=xa, domain=domain, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=fn, text1=text1)

fn = f'{path_results}/OHC/OHC_trend_vert_int_map_rcp-c2'
text1 = 'RCP-C2'
xa = OHC_vert_trend_rcp-OHC_vert_trend_c2
f, ax = map_eq_earth(xa=xa, domain=domain, cmap=cmap, minv=minv, maxv=maxv, label=label, filename=fn, text1=text1)

## zonal integrals

In [ ]:
lats = ctrl.TLAT_bins

f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.84,.82])
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)

y1 = ctrl_S.OHC_zonal.mean(dim='time').values
y2 = 0
S  = plt.fill_between(lats, y1, y2, color='C3', alpha=.5, label='Southern')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal).mean(dim='time').values
y2 = np.nan_to_num(ctrl_S.OHC_zonal.mean(dim='time').values)
A  = plt.fill_between(lats, y1, y2, color='C0', alpha=.5, label='Atlantic')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal).mean(dim='time').values
y2 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal                 ).mean(dim='time').values
P  = plt.fill_between(lats, y1, y2, color='C1', alpha=.5, label='Pacific')

y1 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal+ctrl_I.OHC_zonal).mean(dim='time').values
y2 = (np.nan_to_num(ctrl_S.OHC_zonal)+ctrl_A.OHC_zonal+ctrl_P.OHC_zonal                 ).mean(dim='time').values
I  = plt.fill_between(lats, y1, y2, color='C2', alpha=.5, label='Indian')

y1 = (ctrl_A.OHC_zonal+ctrl_P.OHC_zonal+ctrl_M.OHC_zonal).mean(dim='time').values
y2 = (ctrl_A.OHC_zonal+ctrl_P.OHC_zonal                 ).mean(dim='time').values
M  = plt.fill_between(lats, y1, y2, color='C4', alpha=.5, label='Mediterranean')

G, = plt.plot(lats , ctrl.OHC_zonal.mean(dim='time').values, c='k', lw=3, label='global integral')

plt.text(.98,.9, f'CTRL mean', ha='right', transform=ax.transAxes, fontsize=16)

plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.ylim((0,2.6e18))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC integral [J/m]', fontsize=16)
plt.legend(handles=[G, M, S, A, P, I], fontsize=16, loc=2, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_mean')

In [ ]:
ctrl_zonal_trend_G = zonal_trend(ctrl)
ctrl_zonal_trend_A = zonal_trend(ctrl_A)
ctrl_zonal_trend_P = zonal_trend(ctrl_P)
ctrl_zonal_trend_I = zonal_trend(ctrl_I)
ctrl_zonal_trend_M = zonal_trend(ctrl_M)
ctrl_zonal_trend_S = zonal_trend(ctrl_S)

rcp_zonal_trend_G  = zonal_trend(rcp)
rcp_zonal_trend_A  = zonal_trend(rcp_A)
rcp_zonal_trend_P  = zonal_trend(rcp_P)
rcp_zonal_trend_I  = zonal_trend(rcp_I)
rcp_zonal_trend_M  = zonal_trend(rcp_M)
rcp_zonal_trend_S  = zonal_trend(rcp_S)

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(ctrl_zonal_trend_G.TLAT_bins, ctrl_zonal_trend_G, c='k' , lw=3  , ls='--', label='CTRL global')
plt.plot(ctrl_zonal_trend_A.TLAT_bins, ctrl_zonal_trend_A, c='C0', lw=1.5, ls='--', label='CTRL Atlantic')
plt.plot(ctrl_zonal_trend_P.TLAT_bins, ctrl_zonal_trend_P, c='C1', lw=1.5, ls='--', label='CTRL Pacific')
plt.plot(ctrl_zonal_trend_I.TLAT_bins, ctrl_zonal_trend_I, c='C2', lw=1.5, ls='--', label='CTRL Indian')
plt.plot(ctrl_zonal_trend_S.TLAT_bins, ctrl_zonal_trend_S, c='C3', lw=1.5, ls='--', label='CTRL Southern')
plt.plot(ctrl_zonal_trend_M.TLAT_bins, ctrl_zonal_trend_M, c='C4', lw=1.5, ls='--', label='CTRL Mediterr.')

plt.plot(rcp_zonal_trend_G.TLAT_bins , rcp_zonal_trend_G , c='k' , lw=3  , ls='-' , label='RCP global')
plt.plot(rcp_zonal_trend_A.TLAT_bins , rcp_zonal_trend_A , c='C0', lw=1.5, ls='-' , label='RCP Atlantic')
plt.plot(rcp_zonal_trend_P.TLAT_bins , rcp_zonal_trend_P , c='C1', lw=1.5, ls='-' , label='RCP Pacific')
plt.plot(rcp_zonal_trend_I.TLAT_bins , rcp_zonal_trend_I , c='C2', lw=1.5, ls='-' , label='RCP Indian')
plt.plot(rcp_zonal_trend_S.TLAT_bins , rcp_zonal_trend_S , c='C3', lw=1.5, ls='-' , label='RCP Southern')
plt.plot(rcp_zonal_trend_M.TLAT_bins , rcp_zonal_trend_M , c='C4', lw=1.5, ls='-' , label='RCP Mediterr.')
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('zonal OHC linear trend [J/m/yr]', fontsize=16)
plt.legend(fontsize=12, ncol=2, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_trend')

## latitude - depth

In [ ]:
ctrl_zonal_levels_trend_G = zonal_levels_trend(ctrl)
ctrl_zonal_levels_trend_A = zonal_levels_trend(ctrl_A)
ctrl_zonal_levels_trend_P = zonal_levels_trend(ctrl_P)
ctrl_zonal_levels_trend_I = zonal_levels_trend(ctrl_I)
ctrl_zonal_levels_trend_M = zonal_levels_trend(ctrl_M)
ctrl_zonal_levels_trend_S = zonal_levels_trend(ctrl_S)

rcp_zonal_levels_trend_G  = zonal_levels_trend(rcp)
rcp_zonal_levels_trend_A  = zonal_levels_trend(rcp_A)
rcp_zonal_levels_trend_P  = zonal_levels_trend(rcp_P)
rcp_zonal_levels_trend_I  = zonal_levels_trend(rcp_I)
rcp_zonal_levels_trend_M  = zonal_levels_trend(rcp_M)
rcp_zonal_levels_trend_S  = zonal_levels_trend(rcp_S)

In [ ]:
rcp_zonal_levels_trend_I.plot()

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_ctrl, Y_ctrl = np.meshgrid(ctrl.TLAT_bins[11:], -tdepth/1e3)
maxv = 7e11
plt.pcolormesh(X_ctrl, Y_ctrl, ctrl_zonal_levels_trend_G.T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar()
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_ctrl')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 7e11
plt.pcolormesh(X_rcp, Y_rcp, rcp_zonal_levels_trend_G.T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar(extend='max')
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins[11:], -tdepth/1e3)
maxv = 10e11
plt.pcolormesh(X_rcp, Y_rcp, (rcp_zonal_levels_trend_G-ctrl_zonal_levels_trend_G).T, cmap=cmocean.cm.balance, vmin=-maxv, vmax=maxv)
plt.xticks(np.arange(-90,91,30))
plt.xlim((-92,92))
plt.xlabel('latitude', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
plt.colorbar(extend='max')
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_zonal_levels_trend_rcp-ctrl')

In [ ]:
rcp_P.OHC_zonal_levels.mean(dim='time').plot()

In [ ]:
P_lat_0_100   = rcp_P.OHC_zonal_levels.sel({'z_t':slice(0,1e5)}).sum(dim='z_t')
P_lat_100_300 = rcp_P.OHC_zonal_levels.sel({'z_t':slice(1e5,3e5)}).sum(dim='z_t')

In [ ]:
rcp_times

In [ ]:
plt.plot(np.polyfit(rcp_times, P_lat_0_100, 1)[0])
plt.plot(np.polyfit(rcp_times, P_lat_100_300, 1)[0])

In [ ]:
P_lat_0_100_mean_trend   = np.polyfit(rcp_times, P_lat_0_100, 1)[0]
P_lat_100_300_mean_trend = np.polyfit(rcp_times, P_lat_100_300, 1)[0]
P_lat_0_100_mean_trend   = np.mean(P_lat_0_100_mean_trend[np.nonzero(P_lat_0_100_mean_trend)])
P_lat_100_300_mean_trend = np.mean(P_lat_100_300_mean_trend[np.nonzero(P_lat_100_300_mean_trend)])
print(f'{P_lat_0_100_mean_trend:.2e}, {P_lat_100_300_mean_trend:.2e}')

In [ ]:
P_lat_0_100_detr   = (P_lat_0_100   - P_lat_0_100.sel(  {'time':slice(730000,735000)}).mean(dim='time')).where(P_lat_0_100!=0)
P_lat_100_300_detr = (P_lat_100_300 - P_lat_100_300.sel({'time':slice(730000,735000)}).mean(dim='time')).where(P_lat_100_300!=0)
for t in range(len(rcp_times)):
    P_lat_0_100_detr[t,:]   =  P_lat_0_100_detr[t,:]   - P_lat_0_100_mean_trend*t
    P_lat_100_300_detr[t,:] =  P_lat_100_300_detr[t,:] - P_lat_100_300_mean_trend*t

In [ ]:
X, Y = np.meshgrid(rcp_times, P_lat_100_300_detr.TLAT_bins)

f, ax = plt.subplots(2,1,figsize=(8,8), sharex=True)

ax[0].pcolormesh(X, Y, P_lat_0_100_detr.T  , cmap=cmocean.cm.balance)
ax[1].pcolormesh(X, Y, P_lat_100_300_detr.T, cmap=cmocean.cm.balance)

ax[1].set_xlabel('time [years]', fontsize=16)
ax[0].text(.02, .9, '0-100 m'  , transform=ax[0].transAxes, fontsize=16)
ax[1].text(.02, .9, '100-300 m', transform=ax[1].transAxes, fontsize=16)
for i in range(2):
    ax[i].tick_params(labelsize=14)
    ax[i].set_ylim((-40,70))
    ax[i].set_ylabel('latitude', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/OHC/OHC_Hovmoeller_lat_Pacific_rcp')

In [ ]:
P_lat_0_100.sel({'TLAT_bins':-5}, method='nearest').plot()


In [ ]:
P_lat_100_300.sel({'TLAT_bins':-5}, method='nearest').plot()

In [ ]:
P_lat_100_300.z_t

In [ ]:
# hiatus
X_rcp, Y_rcp = np.meshgrid(rcp.TLAT_bins, -tdepth[:20]/1e3)
maxv=8e13
plt.pcolormesh(X_rcp, Y_rcp, (rcp.OHC_zonal_levels[28,:20,:]-rcp.OHC_zonal_levels[20,:20,:]), cmap='RdBu_r', vmin=-maxv, vmax=maxv)
plt.colorbar()
plt.tight_layout()